In [185]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [186]:
df=pd.read_csv('generated_data.csv')

In [187]:
# Import the necessary modules
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

In [197]:
df["receiverId"]=df["receiverId"].fillna(0) # a sign bit, in order to distinguish

In [198]:
X = df.loc[:, ~df.columns.isin(["isSucceeded"])]
y = df["isSucceeded"]
from sklearn.model_selection import TimeSeriesSplit
def time_series_split(X,y):
    ts_split = TimeSeriesSplit(n_splits=2)

    # Define the features and target variable
    
    for train_index, test_index in ts_split.split(df):
    # Get the training and validation sets
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train,y_test=y[train_index],y[test_index]

        df_train, df_test = df.iloc[train_index,:], df.iloc[test_index,:]
    return X_train, X_test, y_train, y_test,df_train, df_test

In [199]:
# note: some feature are generated for example grouby should seperate from train and test dataset

# Split the data into training and test sets
X_train, X_test, y_train, y_test,df_train, df_test= time_series_split(X, y)

# not so good to split time series data like this way->done, using TimeSeriesSplit->done

In [206]:
# the receivedId also have an influence, but if has a receiverId, then the pass rate is 100%, so calcuated pass rate for each receiverId is useless as below
# useless -> done

# def added_X_train_test_receiver(df_train,X_train,X_test):
#     data_dict=df_train.groupby(['receiverId']).mean('isSucceeded')['isSucceeded'].to_dict()

#     def add_receiver_rate_train(df): # note player_id and success_rate is one vs one, have no information, but will have information on the test data!!!
#         df['succeed_rate_rec']=df['receiverId'].map(lambda x:data_dict[x])
#         return df

#     def use_lambda(x):
#         import math
#         if x==0: return np.mean(list(data_dict.values()))
#         elif x in data_dict.keys(): # althrough keys() has x==0
#             return data_dict[x]
#         else: return np.mean(list(data_dict.values()))
#     def add_receiver_rate_test(df):
#         df['succeed_rate_rec']=df['receiverId'].map(lambda x:use_lambda(x))
#         return df

#     return add_receiver_rate_train(X_train),add_receiver_rate_test(X_test)
    
# X_train,X_test=added_X_train_test_receiver(df_train,X_train,X_test)
# X_train.succeed_rate_rec.unique() # for test the conclusion above

/var/folders/ws/p_92kz8j46lgdkjkzzrm9wm40000gn/T/ipykernel_39605/1650385110.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['succeed_rate_rec']=df['receiverId'].map(lambda x:data_dict[x])
/var/folders/ws/p_92kz8j46lgdkjkzzrm9wm40000gn/T/ipykernel_39605/1650385110.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['succeed_rate_rec']=df['receiverId'].map(lambda x:use_lambda(x))


In [226]:
# count pair (receiverId and playId)

def added_X_train_test_pair(df_train,X_train,X_test):
    data_dict=df_train.groupby(['receiverId','Player_id']).sum('isSucceeded')['isSucceeded'].to_dict()

    def use_lambda(df):
        import math
        if (df["receiverId"],df['Player_id']) in data_dict.keys():
            return data_dict[(df["receiverId"],df['Player_id'])]
        return 0

    def add_pair_rate_train(df): 
        df['pair_count']=df.apply(use_lambda,axis=1)
        return df

    def add_pair_rate_test(df):
        df['pair_count']=df.apply(use_lambda,axis=1)
        return df

    return add_pair_rate_train(X_train),add_pair_rate_test(X_test)
    
X_train,X_test=added_X_train_test_pair(df_train,X_train,X_test)

# X_test.pair_count.sum() # result is more than 200, so a good idea at least

/var/folders/ws/p_92kz8j46lgdkjkzzrm9wm40000gn/T/ipykernel_39605/3466680499.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pair_count']=df.apply(use_lambda,axis=1)
/var/folders/ws/p_92kz8j46lgdkjkzzrm9wm40000gn/T/ipykernel_39605/3466680499.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pair_count']=df.apply(use_lambda,axis=1)


In [179]:
df # notice Unnamed:0 is to be processed

,Unnamed: 0,posX_passer,posY_passer,received_PosX,received_PosY,isForward,isSucceeded,receiverId,Player_id,Team,...,pass_day,pass_hour,pass_minute,pass_second,start_day,start_hour,start_minute,time_process_hour,time_process_minute,time_process_second
0,0,32,58,26,55,1,1,95583.0,95582,0,...,5,11,39,15,5,11,36,0,3,2
1,1,16,23,20,29,0,1,95601.0,95582,0,...,5,11,39,59,5,11,36,0,3,46
2,2,92,56,68,52,1,1,95597.0,95582,0,...,5,11,41,47,5,11,36,0,5,34
3,3,72,55,76,62,0,1,95587.0,95582,0,...,5,11,45,55,5,11,36,0,9,42
4,4,74,53,83,50,0,1,95579.0,95582,0,...,5,11,46,1,5,11,36,0,9,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,1137,3,8,9,9,0,1,95583.0,95597,1,...,12,12,51,49,12,12,6,0,45,36
1138,1138,4,12,14,47,0,1,95601.0,95597,1,...,12,12,51,52,12,12,6,0,45,39
1139,1139,35,48,35,40,0,1,95617.0,95597,1,...,12,12,52,37,12,12,6,0,46,24
1140,1140,47,58,47,59,0,1,95603.0,95597,1,...,12,12,53,57,12,12,6,0,47,44


In [227]:
data_dict=df_train.groupby(['Player_id']).mean('isSucceeded')['isSucceeded'].to_dict()

In [228]:
def add_success_rate_train(df): # note player_id and success_rate is one vs one, have no information, but will have information on the test data!!!
    df['succeed_rate']=df['Player_id'].map(lambda x:data_dict[x])
    return df


In [229]:
def use_lambda(x):
    import math
    if x in data_dict.keys():
        return data_dict[x]
    return np.mean(list(data_dict.values()))

In [230]:
def add_success_rate_test(df):
    df['succeed_rate']=df['Player_id'].map(lambda x:use_lambda(x))
    return df

In [231]:
X_train=add_success_rate_train(X_train)
X_test=add_success_rate_test(X_test)

/var/folders/ws/p_92kz8j46lgdkjkzzrm9wm40000gn/T/ipykernel_39605/1914586036.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['succeed_rate']=df['Player_id'].map(lambda x:data_dict[x])
/var/folders/ws/p_92kz8j46lgdkjkzzrm9wm40000gn/T/ipykernel_39605/2144164121.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['succeed_rate']=df['Player_id'].map(lambda x:use_lambda(x))


In [232]:
# generate feature based on received_id and also zone
zone_dict=df_train.groupby(['Player_id','Zone']).mean('isSucceeded')['isSucceeded'].to_dict()

In [233]:
# df['zone_rate']=df[['Player_id','Zone']].map(lambda x:zone_dict[(x[0],x[1])]) this can't work, map can only used in series, use apply instead
def use_zone_lambda_train(df):
    return zone_dict[(df["Player_id"],df['Zone'])]
def add_zone_rate_train(df): # note player_id and success_rate is one vs one, have no information, but will have information on the test data!!!
    df['zone_rate']=df[['Player_id','Zone']].apply(use_zone_lambda_train,axis=1)
    return df

def use_zone_lambda_test(df):
    import math
    if (df["Player_id"],df['Zone']) in zone_dict.keys():
        return zone_dict[(df["Player_id"],df['Zone'])]
    else:
        sub_dict = {key: value for key, value in zone_dict.items() if key[1]== df['Zone']}
        return np.mean(list(sub_dict.values()))
def add_zone_rate_test(df):
    df['zone_rate']=df[['Player_id','Zone']].apply(use_zone_lambda_test,axis=1)
    return df

In [234]:
X_train=add_zone_rate_train(X_train)
X_test=add_zone_rate_test(X_test)

/var/folders/ws/p_92kz8j46lgdkjkzzrm9wm40000gn/T/ipykernel_39605/3936894386.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['zone_rate']=df[['Player_id','Zone']].apply(use_zone_lambda_train,axis=1)
/var/folders/ws/p_92kz8j46lgdkjkzzrm9wm40000gn/T/ipykernel_39605/3936894386.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['zone_rate']=df[['Player_id','Zone']].apply(use_zone_lambda_test,axis=1)


In [235]:
def get_dummy_score(X_train, X_test, y_train, y_test):
    from sklearn.dummy import DummyClassifier


    # Create an instance of the DummyClassifier class
    dummy_classifier = DummyClassifier(strategy='most_frequent')

    # Fit the DummyClassifier instance to the training data
    dummy_classifier.fit(X_train, y_train)

    # Evaluate the performance of the DummyClassifier on the test data
    accuracy = dummy_classifier.score(X_test, y_test)

    print("accuracy=",accuracy)

In [236]:
get_dummy_score(X_train, X_test, y_train, y_test)

accuracy= 0.718421052631579


In [237]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
def over_sampler(X,y):
    # Create an instance of the RandomOverSampler class
    random_over_sampler = RandomOverSampler()

    # Create an instance of the SMOTE class
    smote = SMOTE()

    # Fit the RandomOverSampler to the data
    X_resampled, y_resampled = random_over_sampler.fit_resample(X, y)

    # Fit the SMOTE to the data
    X,y = smote.fit_resample(X_resampled, y_resampled)
    return X,y

In [238]:
X,y=over_sampler(X,y)

In [241]:
get_dummy_score(X_train, X_test, y_train, y_test)

accuracy= 0.718421052631579


In [248]:
X_train=X_train.drop(['receiverId'],axis=1)
X_test=X_test.drop(['receiverId'],axis=1)

In [249]:
# Import the necessary libraries and models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# insight: classifiers are just above the dummy model or even above, try to extract more features

# Define a dictionary of classification models
models = {
    "logistic_regression": LogisticRegression(),
    "support_vector_machine": SVC(),
    "k_nearest_neighbors": KNeighborsClassifier(),
    "decision_tree": DecisionTreeClassifier(),
    "random_forest": RandomForestClassifier(),
    "ada_boost": AdaBoostClassifier(),
    "gradient_boosting": GradientBoostingClassifier(),
    "xg_boost": XGBClassifier(),
    "bagging": BaggingClassifier(),
    "extra_trees": ExtraTreesClassifier(),
    "mlp": MLPClassifier(),
    "gaussian_process": GaussianProcessClassifier(),
    "quadratic_discriminant_analysis": QuadraticDiscriminantAnalysis()
}


In [250]:
# Loop through the models and evaluate each one
for name, model in models.items():

    print(name,":")
    model.fit(X_train,y_train)
    print(model.score(X_test,y_test))

logistic_regression :


/Users/horus_liang/opt/anaconda3/envs/intro_to_ds/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6026315789473684
support_vector_machine :
0.718421052631579
k_nearest_neighbors :
0.6921052631578948
decision_tree :
0.718421052631579
random_forest :
0.7605263157894737
ada_boost :
0.5026315789473684
gradient_boosting :
0.718421052631579
xg_boost :
0.718421052631579
bagging :
0.718421052631579
extra_trees :
0.718421052631579
mlp :
0.718421052631579
gaussian_process :
0.29210526315789476
quadratic_discriminant_analysis :
0.718421052631579


/Users/horus_liang/opt/anaconda3/envs/intro_to_ds/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [26]:
# Define the parameter grid
param_grid = {
    'n_estimators': [1000],
    'max_depth': [1,2,3,4,5],
    'min_samples_split': [10,20,30],
    'min_samples_leaf': [5]
}

In [27]:
from sklearn.model_selection import RandomizedSearchCV
# Create an instance of the Random Forest classifier
classifier = RandomForestClassifier()

# Create an instance of the RandomizedSearchCV class
random_search = RandomizedSearchCV(classifier, param_grid)

# Fit the RandomizedSearchCV to the data
random_search.fit(X, y)

# Print the best parameters
print(random_search.best_params_)

{'n_estimators': 1000, 'min_samples_split': 20, 'min_samples_leaf': 5, 'max_depth': 4}


In [28]:
random_search.best_score_

0.7539416226154907

In [29]:
# Extract the best model
classifier = random_search.best_estimator_

In [31]:
from sklearn.feature_selection import SelectFromModel


# Create an instance of the SelectFromModel class
feature_selector = SelectFromModel(classifier)

# Fit the SelectFromModel instance to the data
feature_selector.fit(X, y)

SelectFromModel(estimator=RandomForestClassifier(max_depth=4,
                                                 min_samples_leaf=5,
                                                 min_samples_split=20,
                                                 n_estimators=1000))

In [32]:
# Get the selected features
selected_features = feature_selector.transform(X)

In [34]:
# tempt !!!

from sklearn.model_selection import RandomizedSearchCV
# Create an instance of the Random Forest classifier
classifier = RandomForestClassifier()

# Create an instance of the RandomizedSearchCV class
random_search = RandomizedSearchCV(classifier, param_grid)

# Fit the RandomizedSearchCV to the data
random_search.fit(selected_features, y)

# Print the best parameters
print(random_search.best_params_)

print(random_search.best_score_)

{'n_estimators': 1000, 'min_samples_split': 30, 'min_samples_leaf': 5, 'max_depth': 4}
0.7670956868152915


In [37]:
from imblearn.over_sampling import RandomOverSampler, SMOTE

# Create an instance of the RandomOverSampler class
random_over_sampler = RandomOverSampler()

# Create an instance of the SMOTE class
smote = SMOTE()

# Fit the RandomOverSampler to the data
X_resampled, y_resampled = random_over_sampler.fit_resample(X, y)

# Fit the SMOTE to the data
X_resampled, y_resampled = smote.fit_resample(X, y)


from sklearn.model_selection import RandomizedSearchCV
# Create an instance of the Random Forest classifier
classifier = RandomForestClassifier()

# Create an instance of the RandomizedSearchCV class
random_search = RandomizedSearchCV(classifier, param_grid)

# Fit the RandomizedSearchCV to the data
random_search.fit(X_resampled, y_resampled)

# Print the best parameters
print(random_search.best_params_)

print(random_search.best_score_)


{'n_estimators': 1000, 'min_samples_split': 20, 'min_samples_leaf': 5, 'max_depth': 3}
0.4285341738428922


In [43]:
get_dummy_score(X_resampled, y_resampled)

accuracy= 0.47549019607843135
